# Workstation Heartbeat and Live Data

This example shows how in a separate thread from the term the HeartBeat can be queried with Python. The heartbeat is queried in a separate thread once a second. The HeartBeat represents how many milliseconds it has been since the term has received something from the Thales.

This example also receives the live data. To receive the online display data, the Zahner online display must be switched off.
To do this, the following parameter must be adapted in the file C:/FLINK/usb.ini EnableODisplay=off.

In [9]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode, ThalesRemoteScriptWrapper
import time
import threading

zenniumConnectionLiveData = None

keepThreadRunning = True

# Watch Thread
The following function is used as a thread, in which the HearBeat is queried once a second. The HeartBeat time varies, for example, if EIS is measured at low frequencies, then this time is increased.

The HeartBeat is queried once per second. A timeout of 2 seconds is used to query the HeartBeat. This ensures that the Term responds within 2 seconds, otherwise it can be assumed that the Term software has crashed.

In [10]:
def watchThreadFunction():
    global keepThreadRunning

    zenniumConnection = ThalesRemoteConnection()
    zenniumConnection.connectToTerm("localhost", "Watch")

    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)

    while keepThreadRunning:
        time.sleep(1)
        active = zahnerZennium.getTermIsActive()
        print("active state: " + str(active))
        if active:
            print("beat count: " + str(zahnerZennium.getWorkstationHeartBeat()))
    zenniumConnection.disconnectFromTerm()
    return

# Live Data Thread
The following function is used as a thread which receives the live data instead of the online display.

Only relevant packet types are output to the console. The relevant types are written as comments in the source code.

In [11]:
def liveDataThreadFunction():
    global keepThreadRunning
    global zenniumConnectionLiveData

    print("live thread started")
    while keepThreadRunning:
        try:
            data = zenniumConnectionLiveData.waitForBinaryTelegram()
            packetId = data[0]
            data = data[1:]
            """
            Type:
            1 = Init measurement begin
            2 = Measurement end
            4 = Measurement data names
            5 = Measurement data units
            6 = ASCII data
            """
            if packetId in [1, 2, 4, 5, 6]:
                print(data.decode("ASCII"))
        except:
            """
            The connection to the term has an error or the socket has been closed.
            """
            print("term error live thread")
            keepThreadRunning = False

    print("live thread left")
    return

# Main Program Sequence

In the main program flow, the first thing that happens is that an additional connection to the Term is established with the name "Logging". The live data comes via this connection. For this to be possible, the entry EnableODisplay=off must be set in the file "C:\FLINK\usb.ini". And the actual Zahner Online Display must be closed.

Then the thread is started, which receives the data.

In [12]:
if __name__ == "__main__":
    zenniumConnectionLiveData = ThalesRemoteConnection()
    zenniumConnectionLiveData.connectToTerm("localhost", "Logging")

    liveThread = threading.Thread(target=liveDataThreadFunction)
    liveThread.start()

live thread started


After the connection with the live data, the nominal connection is established, which sends the commands for measurement.

In addition, the [hideWindow()](https://doc.zahner.de/thales_remote/script_wrapper.html#thales_remote.script_wrapper.ThalesRemoteScriptWrapper.hideWindow) method hides the Thales window to protect it from unwanted operation via GUI when controlled remotely.

In [13]:
zenniumConnection = ThalesRemoteConnection()
zenniumConnection.connectToTerm("localhost", "ScriptRemote")

zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
zahnerZennium.forceThalesIntoRemoteScript()
zahnerZennium.hideWindow()

zahnerZennium.calibrateOffsets()

devel version


'OK\r'

The watch thread uses the command interface to the Thales, so it is started after initializing this connection.

In [14]:
watchThread = threading.Thread(target=watchThreadFunction)
watchThread.start()

zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
zahnerZennium.setAmplitude(10e-3)
zahnerZennium.setPotential(0)
zahnerZennium.setLowerFrequencyLimit(750)
zahnerZennium.setStartFrequency(1000)
zahnerZennium.setUpperFrequencyLimit(1500)
zahnerZennium.setLowerNumberOfPeriods(2)
zahnerZennium.setLowerStepsPerDecade(2)
zahnerZennium.setUpperNumberOfPeriods(2)
zahnerZennium.setUpperStepsPerDecade(20)
zahnerZennium.setScanDirection("startToMax")
zahnerZennium.setScanStrategy("single")

zahnerZennium.enablePotentiostat()


zahnerZennium.setFrequency(1)
zahnerZennium.setAmplitude(10e-3)
zahnerZennium.setNumberOfPeriods(3)

print("measurement start")

zahnerZennium.measureEIS()
for i in range(20):
    zahnerZennium.getPotential()
    zahnerZennium.setPotential(0)

print("measurement end")

zahnerZennium.disablePotentiostat()

measurement start
3,Impedance Spectroscopy
devel version
frequency,impedance,phase,time,significance,voltage,current,
Hz,Ohm,rad,s, ,V,A,
active state: True
beat count: 281.0
active state: True
beat count: 187.0
active state: True
beat count: 78.0
active state: True
beat count: 516.0
 1.00940e+03, 1.07426e+06,-1.55002e+00, 0.00000e+00, 9.99000e-01,-2.90489e-05,-2.76135e-12,
active state: True
beat count: 0.0
 1.11450e+03, 9.74292e+05,-1.55009e+00, 2.41300e+00, 1.00000e+00,-2.90489e-05,-2.76135e-12,
 1.23050e+03, 8.82627e+05,-1.55028e+00, 2.77350e+00, 1.00000e+00,-2.90489e-05,-2.76135e-12,
active state: True
beat count: 156.0
 1.35860e+03, 7.99644e+05,-1.54934e+00, 3.20000e+00, 1.00000e+00,-2.90489e-05,-2.76135e-12,
 1.50000e+03, 7.25203e+05,-1.55006e+00, 3.55650e+00, 9.99000e-01,-2.90489e-05,-2.76135e-12,
 1.35860e+03, 7.98927e+05,-1.55026e+00, 3.91350e+00, 9.99000e-01,-2.90489e-05,-2.76135e-12,
active state: True
beat count: 110.0
 1.23050e+03, 8.81829e+05,-1.55050e+00, 4.27150e+00, 9

'OK\r'

Closing the threads and then waiting until they are closed.

Before the connection is disconnected, the Thales window is displayed again.

In [15]:
print("set thread kill flag")
keepThreadRunning = False

print("disconnect connections")
zahnerZennium.showWindow()
zenniumConnection.disconnectFromTerm()
zenniumConnectionLiveData.disconnectFromTerm()

print("join the threads")
liveThread.join()
watchThread.join()

print("finish")

active state: True
beat count: 15.0
set thread kill flag
disconnect connections
term error live thread
live thread left
active state: True
beat count: 78.0
join the threads
finish
